In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端的机器学习：MLOps阶段2：使用TabNet内置算法开始训练表格模型

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_tabnet.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_tabnet.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_tabnet.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示了如何在Vertex AI平台上使用TabNet内置算法服务来训练自定义表格模型。

TabNet结合了两个世界的优点：它是可解释的（类似于简单的基于树的模型），同时又能够获得高性能（类似于深度神经网络）。这使得它非常适用于零售、金融和保险行业的应用，如预测信用评分、欺诈检测和预测。

TabNet使用一种名为顺序注意力的机器学习技术，以选择在模型的每个步骤中要从中推理的模型特征。这种机制使得能够解释模型如何得出其预测，并帮助它学习更准确的模型。TabNet不仅优于其他神经网络和决策树，而且提供可解释的特征归因。

研究论文：[TabNet: 专注可解释的表格学习](https://arxiv.org/pdf/1908.07442.pdf)

### 目标

在这本笔记本中，您将学习如何运行构建的`Vertex AI TabNet`算法，用于训练自定义表格模型。

该教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI TabNet`
- `Vertex AI Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 获取训练数据。
- 为`Vertex AI TabNet`容器配置训练参数。
- 使用 CSV 数据使用`Vertex AI Training`训练模型。
- 将模型上传为`Vertex AI Model`资源。
- 部署`Vertex AI Model`资源到`Vertex AI Endpoint`资源。
- 使用部署的模型进行预测。
- 调整`Vertex AI TabNet`模型的超参数。
- 使用 BigQuery 表格使用`Vertex AI Training`训练模型。

### 数据集

本教程使用了公共云存储桶`gs://cloud-samples-data/ai-platform-unified/datasets/tabular/`中的`petfinder`数据集，该数据集是从[PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction)生成的。该数据集预测动物被领养的速度。

### 费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成一个费用估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade tensorflow -q
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform tensorboard-plugin-profile -q
! gcloud components update --quiet

重新启动内核

在安装完额外的软件包之后，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**不管您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建账户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs、Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并使用以`$`为前缀的Python变量进行插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 `REGION` 变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有的Vertex AI服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能会使用共享的测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐号

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经经过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按照提示进行oAuth身份验证。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**将此服务帐号授予项目访问权限**部分，点击**角色**下拉列表。在筛选框中输入“Vertex AI”，选择**Vertex AI管理员**。在筛选框中输入“Storage Object Admin”，选择**Storage Object Admin**。

5. 点击*创建*。包含您密钥的JSON文件会下载到本地环境中。

6. 在下面的单元格中输入您的服务帐号密钥路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您使用的是什么笔记本环境，都需要完成以下步骤。**

当您在 Python 中初始化 Vertex AI SDK 时，您需要指定一个云存储临时桶。这个临时桶是您数据集和模型资源中所有数据在各个会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

导入库并定义常量###

In [ ]:
import os
from datetime import datetime

import google.cloud.aiplatform as aip

%load_ext tensorboard

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU和分配给虚拟机实例的GPU数量的容器映像。例如，要使用一个GPU容器映像，将4个Nvidia Telsa K80 GPU分配给每个虚拟机，您可以指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用运行在CPU上的容器映像。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：早于2.3版本的TF GPU支持将无法加载本教程中的自定义模型。这是一个已知问题，并在TF 2.3中得到解决。这是由于在服务函数中生成的静态图操作所造成的。如果您在自己的自定义模型上遇到此问题，请使用具有GPU支持的TF 2.3容器映像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

DEPLOY_GPU, DEPLOY_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的计算资源。
  - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
  - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下内容不支持用于训练：*

  - `standard`：2个vCPUs
  - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

#### 设置训练容器

接下来，您将使用预构建的`Vertex AI TabNet`容器进行模型训练。

In [ ]:
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/tab_net_v2"

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

为部署设置预构建容器

为预测设置预构建的Docker容器映像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

## 获取训练数据

首先，您从公共云存储桶获取训练数据副本（以CSV文件格式），然后将训练数据复制到您的云存储桶中。

In [ ]:
# Please note that if you use csv input, the first column is the label column.

IMPORT_FILE = "petfinder-tabular-classification-tabnet-with-header.csv"
TRAINING_DATA_PATH = f"{BUCKET_URI}/data/petfinder/train.csv"

! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {TRAINING_DATA_PATH}

### 创建并运行 `Vertex AI TabNet` 训练作业

要训练一个自定义的 TabNet 模型，您需要执行两个步骤：1）创建一个自定义训练作业，2）运行这个作业。

#### 创建自定义训练作业

使用 `CustomTrainingJob` 类创建一个自定义训练作业，并设置以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：用于训练的容器镜像。
- `model_serving_container_image_uri`：能够为您的模型提供预测的容器的 URI — 可以是预先构建的容器图片。

In [ ]:
DATASET_NAME = "petfinder"  # Change to your dataset name.

job = aip.CustomContainerTrainingJob(
    display_name=f"{DATASET_NAME}_{TIMESTAMP}",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

print(job)

配置TabNet训练的参数设置

以下表格显示了TabNet训练作业的参数：

| 参数 | 数据类型 | 描述 | 是否必填 |
|--|--|--|--|
| `preprocess` | 布尔参数 | 指定此参数以启用自动预处理。 | 否 |
| `job_dir` | 字符串 | 存储模型输出文件的云存储目录。 | 是 |
| `input_metadata_path` | 字符串 | 用于训练数据集的TabNet特定元数据的GCS路径。请参阅上面如何创建元数据。 | 否 |
| `training_data_path` | 字符串 | 存储训练数据的云存储模式。 | 是 |
| `validation_data_path` | 字符串 | 存储评估数据的云存储模式。 | 否 |
| `test_data_path` | 字符串 | 存储测试数据的云存储模式。 | 是 |
| `input_type` | 字符串 | “bigquery”或“csv”-输入表格数据的类型。如果提到csv，则第一列被视为目标。如果CSV文件有标题，还要传递标志“data_has_header”。如果使用“bigquery”，可以提供训练/验证数据路径，也可以提供BigQuery项目、数据集和表名以便预处理生成训练和验证数据集。 | 是 |
| `model_type` | 字符串 | 学习任务类型，如分类或回归。 | 是 |
| `split_column` | 字符串 | 用于创建训练、验证和测试拆分的列名。列的值（也称为table['split_column']）应为“TRAIN”、“VALIDATE”或“TEST”。对于bigquery输入有效。 | 否 |
| `train_batch_size` | 整数 | 训练的批量大小。 | 否 - 默认为1024。 |
| `eval_split` | 浮点数 | 用于评估数据集的分割比例，如果未提供`validation_data_path`。 | 否 - 默认为0.2 |
| `learning_rate` | 浮点数 | 训练的学习率。 | 否 - 默认为指定优化器的默认学习率。 |
| `eval_frequency_secs` | 整数 | 评估和保存检查点进行的频率。默认为600。 | 否 |
| `num_parallel_reads` | 整数 | 用于读取输入文件的线程数量。我们建议设置为等于或略小于机器的CPU数量，以获得最佳性能。例如，每个GPU的默认选择为6。 | 是。 |
| `optimizer` | 字符串 | 训练优化器。支持任何TF2.3 Keras优化器的小写字符串名称（'sgd'、'adam'、'ftrl'等）。请参阅[TensorFlow文档](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)。 | 否 - 默认为'adam'。 |
| `data_cache` | 字符串 | 选择“memory”、“disk”或“no_cache”来缓存数据。对于大型数据集，将数据缓存到内存可能会导致内存溢出错误，因此建议选择“disk”。您可以在配置文件中指定磁盘大小（如下例所示）。确保请求一个足够大（例如TB大小）的磁盘来写入数据，对于大型（B级别）数据集。 | 否，默认为“memory”。 |
| `bq_project` | 字符串 | BigQuery项目的名称。如果input_type=bigquery并且使用标志--preprocessing，则需要此参数。这是指定训练、验证和测试数据路径的另一种选择。 | 否。 |
| `dataset_name` | 字符串 | BigQuery数据集的名称。如果input_type=bigquery并且使用标志--preprocessing，则需要此参数。这是指定训练、验证和测试数据路径的另一种选择。 | 否。 |
| `table_name` | 字符串 | BigQuery表的名称。如果input_type=bigquery并且使用标志--preprocessing，则需要此参数。这是指定训练、验证和测试数据路径的另一种选择。 | 否。 |
| `loss_function_type` | 字符串 | TabNet中有几种损失函数类型。对于回归：包括mse/mae。对于分类：包括cross_entropy/weighted_cross_entropy/focal_loss。 | 否。如果值为"default"，我们对于回归使用mse，对于分类使用cross_entropy。 |
| `deterministic_data` | 布尔参数 | 从表格数据中读取时的确定性。默认设置为False。当设置为True时，实验是确定性的。为了在大型数据集上进行快速训练，我们建议将deterministic_data设置为False，尽管结果中有随机性（在大型数据集上变得可以忽略不计）。请注意，由于map-reduce引起的随机性，在分布式训练中仍然不保证确定性，因为代数操作的顺序具有有限精度。然而，在实践中，这在大型数据集上是可以忽略的。对于希望100%确定性的情况，除了将deterministic_data设置为True之外，我们建议使用单个GPU进行训练（例如使用MACHINE_TYPE="n1-highmem-8"）。| 否，默认为False。 |
| `stream_inputs` | 布尔参数 | 从GCS实时流式传输输入数据，而不是在本地下载 - 该选项建议用于快速运行时。 | 否。 |
| `large_category_dim` | 整数 | 嵌入的维度 - 如果分类列的不同类别数大于large_category_thresh，则我们使用一个large_category_dim维的嵌入，而不是1维嵌入。默认为1。我们建议增加（例如在大多数情况下增加到~5，如果数据集中一般类别数极大，则增加到~10），如果提高准确性是主要目标，而不是计算效率和可解释性。 | 否。 |
| `large_category_thresh` | 整数 | 分类列基数的阈值 - 如果分类列的不同类别数大于large_category_thresh，则我们使用一个large_category_dim维的嵌入，而不是1维嵌入。默认为300。建议减小它（例如减小到约10），如果提高准确性是主要目标，而不是计算效率和可解释性。 | 否。 |
| `yeo_johnson_transform` | 布尔参数 | 启用可训练的Yeo-Johnson Power Transform（默认情况下未启用）。请参阅此链接：https://www.stat.umn.edu/arc/yjpower.pdf 了解更多关于Yeo-Johnson Power Transform的信息。通过我们的实现，转换参数可以与TabNet一起学习，以端对端的方式进行训练。 | 否。 |
| `apply_log_transform` | 布尔参数 | 如果元数据中包含对数变换统计信息，并且此标志为true，则输入特征将被对数变换。使用false表示不使用变换，true（默认）表示使用。对于具有偏斜数值分布的数据集，对数变换可能非常有帮助。 | 否。 |
| `apply_quantile_transform` | 布尔参数 | 如果元数据中包含分位数统计信息，并且此标志为true，则输入特征将被分位数转换。使用false表示不使用变换，true（默认）表示使用。目前支持BigQuery input_type。 | 否。 |
| `replace_transformed_features` | 布尔参数 | 如果为true，则如果对特征应用变换，则该特征将被替换。如果为false（默认选项），则转换的特征将作为新特征添加到特征列列表中。使用true替换特征为变换，使用false将转换的特征附加为新列。 | 否。 |
| `target_column` | 字符串 | 标签列的名称。请注意，对于分类，标签需要为字符串或整数类型。 | 否。 |
| `prediction_raw_inputs` | 布尔参数 | 如果设置此参数，模型服务允许我们将特征作为张量字典传递，而不是CSV行。 | 否。 |
| `exclude_key` | 布尔参数 | 如果设置此参数，我们将在输入/输出中排除一个键。该键对于批预测过程中的输入很有帮助，并且以不可预测的顺序保存输出。该键帮助将输出与输入匹配。 | 否。 |

了解更多关于[开始使用内置TabNet算法](https://cloud.google.com/ai-platform/training/docs/algorithms/tab-net-start)。

In [ ]:
ALGORITHM = "tabnet"
MODEL_TYPE = "classification"
MODEL_NAME = f"{DATASET_NAME}_{ALGORITHM}_{MODEL_TYPE}"

OUTPUT_DIR = f"{BUCKET_URI}/{MODEL_NAME}_{TIMESTAMP}"
print("Output dir: ", OUTPUT_DIR)

CMDARGS = [
    "--preprocess",
    "--data_has_header",
    f"--training_data_path={TRAINING_DATA_PATH}",
    f"--job-dir={OUTPUT_DIR}",
    f"--model_type={MODEL_TYPE}",
    "--max_steps=2000",
    "--batch_size=4096",
    "--learning_rate=0.01",
    "--prediction_raw_inputs",
    "--exclude_key",
]

### 训练TabNet模型

使用`run`方法开始训练，需要以下参数：

- `args`：要传递给TabNet训练容器的命令行参数。
- `replica_count`：工作节点副本的数量。
- `model_display_name`：如果脚本生成一个托管的`Model`，则显示`Model`的名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点副本的加速器数量。

`run`方法创建一个训练流水线，用于训练并创建一个`Model`对象。训练流水线完成后，`run`方法返回`Model`对象。

In [ ]:
MODEL_DIR = OUTPUT_DIR

if TRAIN_GPU:
    model = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        sync=True,
    )
else:
    model = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

print(model.gca_resource)

#### 删除训练作业

使用`delete()`方法来删除训练作业。

In [ ]:
job.delete()

### 部署模型

在使用模型进行预测之前，您需要将其部署到一个`Endpoint`上。您可以通过在`Model`资源上调用`deploy`函数来实现这一点。这将做两件事情：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。
2. 将`Model`资源部署到`Endpoint`资源上。


该函数接受以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：在端点上流量的百分比分配给该模型，作为一个或多个键/值对字典来指定。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中"0"指的是上传的这个模型，100表示100%的流量。
   - 如果端点上已经存在其他模型，并且要分配流量给它们，则使用`model_id`来指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是要部署到端点的现有模型的模型ID。百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作副本的加速器数量。
- `starting_replica_count`：最初要提供的计算实例数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在本教程中，只提供一个实例。

### 流量分配

`traffic_split`参数被指定为一个Python字典。您可以将多个模型实例部署到一个端点上，并设置流量分配给每个实例的百分比。

您可以使用流量分配逐渐将新模型引入生产环境。例如，如果您在生产环境中有一个现有模型占据100%的流量，您可以将一个新模型部署到同一个端点，将10%的流量引导到新模型，并将原始模型的流量降低到90%。这样可以在最小化对大多数用户的干扰的情况下监控新模型的性能。

### 计算实例扩展

您可以指定一个单一实例（或节点）来为您的在线预测请求提供服务。本教程使用一个节点，因此变量`MIN_NODES`和`MAX_NODES`都设置为`1`。

如果想要使用多个节点来为在线预测请求提供服务，请将`MAX_NODES`设置为您想要使用的节点的最大数量。Vertex AI会自动扩展用于提供预测服务的节点数量，直到达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

### Endpoint

该方法将阻塞，直到模型部署完毕并最终返回一个`Endpoint`对象。如果这是第一次将模型部署到端点上，可能需要额外几分钟来完成资源的配额。

In [ ]:
DEPLOYED_NAME = "{DATASET_NAME}_" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_COMPUTE.name,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

获取服务签名

接下来，将模型下载到本地并查询模型的服务签名。 服务签名的形式将为：

    （"feature_name_1"，"feature_name_2"，...）

In [ ]:
import tensorflow as tf

loaded = tf.saved_model.load(MODEL_DIR + "/model")
loaded.signatures

做出预测

最后，您可以使用 `predict()` 方法来做出预测。每个实例都以以下字典格式指定：

    { "feature_name_1": 值, "feature_name_2": 值, ... }

In [ ]:
prediction = endpoint.predict(
    [
        {
            "Age": 3,
            "Breed1": "Tabby",
            "Color1": "Black",
            "Color2": "White",
            "Fee": 100,
            "FurLength": "Short",
            "Gender": "Male",
            "Health": "Healthy",
            "MaturitySize": "Small",
            "PhotoAmt": 2,
            "Sterilized": "No",
            "Type": "Cat",
            "Vaccinated": "No",
        }
    ]
)

print(prediction)

超参数调整

在成功训练模型、部署模型并调用它进行预测之后，您可能希望优化训练过程中使用的超参数，以提高模型的准确性和性能。请查阅 Vertex AI 文档，了解超参数调整的概述以及如何在您的 Vertex 训练作业中使用它。

在此示例中，以下代码运行一个带有4个试验的 Vertex AI 超参数调整作业，试图最大化验证 AUC 指标。它优化的超参数包括最大步数和学习率。

创建试验配置

接下来，您需要构建一个包含超参数试验设置的YAML文件。

In [ ]:
config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: {TRAIN_COMPUTE}
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 100
    containerSpec:
      imageUri: {TRAIN_IMAGE}
      args:
      - --preprocess 
      - --data_has_header
      - --training_data_path={TRAINING_DATA_PATH}
      - --job-dir={OUTPUT_DIR}
      - --batch_size=1028
      - --model_type={MODEL_TYPE}
      - --prediction_raw_inputs
"""

!echo $'{config}' > ./config.yaml

### 执行超参数调整试验

接下来，您可以使用命令`gcloud ai hp-tuning-jobs create`来执行超参数调整作业。

该作业将以异步方式运行。您可以使用`gcloud ai hp-tuning-jobs describe`来轮询作业的状态。

In [ ]:
MAX_TRIAL_COUNT=4
PARALLEL_TRIAL_COUNT=2

output = ! gcloud ai hp-tuning-jobs create \
  --config=config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name={DATASET_NAME}_{TIMESTAMP}

print(output)

DESCRIBE = output[5]
print("Describe cmd:", DESCRIBE)

取消超参数调整作业

接下来，使用命令 `gcloud ai hp-tuning-jobs cancel` 取消超参数调整作业。

In [ ]:
DESCRIBE = output[5]
print("Describe cmd:", DESCRIBE)

args = DESCRIBE.split(" ")
JOB_ID = args[7]

! gcloud ai hp-tuning-jobs cancel {JOB_ID} --region={REGION}

## 使用BigQuery输入创建和运行`Vertex AI TabNet`训练作业

您可以使用来自Cloud Storage位置的CSV输入数据，或者使用BigQuery表训练`Vertex AI TabNet`表格模型。在下一个示例中，您将使用来自BigQuery的数据训练模型。

### 为训练数据创建BigQuery表格。

首先，请使用上述CSV文件创建BQ数据集。

In [ ]:
# Create the BQ dataset
! bq --location={REGION} mk --dataset {PROJECT_ID}:{DATASET_NAME}
# Create the BQ table and populate it with data from the CSV file
! bq --location={REGION} load --source_format=CSV --autodetect {PROJECT_ID}:{DATASET_NAME}.train {BUCKET_URI}/data/petfinder/train.csv

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建自定义训练任务，包括以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `model_serving_container_image_uri`：可以为您的模型提供预测的容器的URI，可以是预构建的。

In [ ]:
job = aip.CustomContainerTrainingJob(
    display_name=f"{DATASET_NAME}_{TIMESTAMP}",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

print(job)

### 配置TabNet训练的参数设置

接下来，配置使用BigQuery输入进行训练的参数设置。

In [ ]:
BQ_PROJECT = f"{PROJECT_ID}"
BQ_TABLE = "train"
TARGET_COLUMN = "Adopted"

CMDARGS = [
    "--eval_frequency_secs=10800",
    "--input_type=bigquery",
    "--preprocess",
    f"--model_type={MODEL_TYPE}",
    "--stream_inputs",
    "--max_steps=3000",
    f"--bq_project={BQ_PROJECT}",
    f"--dataset_name={DATASET_NAME}",
    f"--table_name={BQ_TABLE}",
    f"--target_column={TARGET_COLUMN}",
    "--num_parallel_reads=2",
    "--optimizer_type=adam",
    "--data_cache=disk",
    "--deterministic_data=False",
    "--loss_function_type=weighted_cross_entropy",
    "--replace_transformed_features=True",
    "--apply_quantile_transform=True",
    "--apply_log_transform=True",
    "--batch_size=32768",
    "--learning_rate=0.01",
    "--prediction_raw_inputs",
    "--exclude_key",
    f"--job-dir={OUTPUT_DIR}",
]

### 训练 TabNet 模型

使用 `run` 方法开始训练，该方法接受以下参数：

- `args`：要传递给 TabNet 训练容器的命令行参数。
- `replica_count`：工作器副本的数量。
- `model_display_name`：如果脚本生成管理的 `Model`，则为 `Model` 的显示名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器副本的加速器数量。

`run` 方法创建一个训练流水线，训练并创建一个`Model`对象。训练流水线完成后，`run` 方法会返回`Model`对象。

In [ ]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)


MODEL_DIR = OUTPUT_DIR

if TRAIN_GPU:
    model_bq = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )
else:
    model_bq = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )

print(model_bq.gca_resource)

删除培训作业

使用`delete()`方法来删除培训作业。

In [ ]:
job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

In [ ]:
delete_bucket = False

# Delete BQ table
! bq rm -f {PROJECT_ID}:{DATASET_NAME}.train

try:
    endpoint.undeploy_all()
    endpoint.delete()
    model.delete()
    model_bq.delete()
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI